In [1]:
!pip install pykrx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 59.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.8/259.8 kB 18.4 MB/s eta 0:00:00


## 코스닥에 상장된 주식 종목 코드들 가져오기

In [2]:
from pykrx import stock
import pandas as pd
import numpy as np

In [3]:

# 기업명과 종목코드를 포함한 딕셔너리
company_dic = {
    "SGA": "049470", "SGA솔루션즈": "184230", "안랩": "053800", "시큐브": "131090", "윈스": "136540", "이글루": "067920",
    "한컴위드": "054920", "네오리진": "094860", "케이사인": "192250", "이스트소프트": "047560", "라온시큐어": "042510",
    "파수": "150900", "한국정보인증": "053300", "지란지교시큐리티": "208350", "수산아이앤티": "050960", "지니언스": "263860",
    "드림시큐리티": "203650", "모니터랩": "434480", "파이오링크": "170790", "시큐센": "232830", "시큐레터": "418250",
    "다우기술": "023590", "다우데이타": "032190", "한글과컴퓨터": "030520", "MDS테크": "086960", "아이티센": "124500",
    "콤텍시스템": "031820", "소프트센": "032680", "대신정보통신": "020180", "쌍용정보통신": "010280", "에스넷": "038680"
}

# 🔹 조회할 날짜 입력
target_date = input("조회할 날짜를 입력하세요 (YYYYMMDD): ").strip()

# 🔹 전일 날짜 계산
prev_date = stock.get_nearest_business_day_in_a_week(target_date, prev=True)  # 전 거래일 찾기

# 🔹 결과 저장할 리스트
data_list = []

# 🔹 모든 종목에 대해 데이터 가져오기
for company, code in company_dic.items():
    try:
        # 🔹 등락률이 유지되도록 전일 + target_date 포함하여 가져옴
        df_price = stock.get_market_ohlcv_by_date(prev_date, target_date, code, adjusted=False)

        # 🔹 시가총액 데이터 가져오기
        df_market_cap = stock.get_market_cap_by_date(prev_date, target_date, code)
        # 안전한 형변환 (int64로 변환)
        df_price["거래량"] = df_price["거래량"].astype("int64")
        df_price["종가"] = df_price["종가"].astype("int64")

        # 🔹 시가총액 추가
        df_price["시가총액"] = df_market_cap["시가총액"]

        # 🔹 종목명 추가
        df_price["기업명"] = company
        df_price["종목코드"] = code


        # 🔹 target_date의 데이터만 선택하여 저장 (등락률 유지됨)
        data_list.append(df_price.loc[target_date])

    except Exception as e:
        print(f"⚠ {company}({code}) 데이터 수집 실패: {e}")

# 🔹 최종 데이터프레임 생성
df_result = pd.DataFrame(data_list).reset_index()

# 🔹 컬럼 정리
df_result = df_result[["기업명", "종목코드", "시가", "고가", "저가", "종가", "거래량", "거래대금", "등락률", "시가총액"]]
df_result['전날 종가'] = df_result['종가'] / (1 + (df_result['등락률'] / 100))
df_result['전일대비'] = df_result['종가'] - df_result['전날 종가']
df_result = df_result.rename(columns={'기업명': '회사', '시가총액': '시가총액(억)', '거래대금': '거래대금(천)'})
df_result['등락률'] = df_result['등락률'].astype(str) + '%'
df_result['날짜'] = target_date[:4] + "." + target_date[4:6] + "." + target_date[6:]
df_result['시가총액(억)'] = df_result['시가총액(억)'] / 100000000
df_result['시가총액(억)'] = df_result['시가총액(억)'].round(0)
df_result['시가총액(억)'] = df_result['시가총액(억)'].astype(int)
df_result['거래대금(천)'] = df_result['거래대금(천)'] // 1000000
# df_result['거래대금(천)'] = df_result['거래대금(천)'].round(0)
# df_result['거래대금(천)'] = df_result['거래대금(천)'].astype(int)
display(df_result)
# 회사,  종가,, 전일대비, 등략률, 시가, 고가, 저가, 거래량, 거래대금, 시가총액(억),

df_num = df_result[['회사', '종가', '전일대비', '등락률', '시가', '고가', '저가', '거래량', '거래대금(천)', '시가총액(억)', '날짜']]
voiceye_row = {
    "회사": "SGA임베디드*(코스닥 종목)",
    "종가": 2965,
    "전일대비": 0,
    "등락률":0,
    "시가": 2965,
    "고가": 2965,
    "저가": 2965,
    "거래량": 0,
    "거래대금(천)": 0,
    "시가총액(억)": 161,
    "날짜": "-"

}

jasa = df_num.iloc[:2]
elsee = df_num.iloc[2:]
df_num = pd.concat([jasa, pd.DataFrame([voiceye_row]),elsee], ignore_index=True)
display(df_num)



조회할 날짜를 입력하세요 (YYYYMMDD): 20250131


,회사,종목코드,시가,고가,저가,종가,거래량,거래대금(천),등락률,시가총액(억),전날 종가,전일대비,날짜
0,SGA,049470,317,325,315,315,48187,15,0.64%,185,312.996824,2.003176,2025.01.31
1,SGA솔루션즈,184230,484,504,484,492,60081,29,0.61%,308,489.016968,2.983032,2025.01.31
2,안랩,053800,71200,78300,71200,76100,293959,22360,6.73%,8467,71301.417321,4798.582679,2025.01.31
3,시큐브,131090,911,927,909,920,110474,101,0.55%,392,914.967715,5.032285,2025.01.31
4,윈스,136540,11370,11470,11190,11400,19764,224,0.26%,1400,11370.437379,29.562621,2025.01.31
5,이글루,067920,5080,5320,4985,5090,91341,466,0.2%,560,5079.840450,10.159550,2025.01.31
6,한컴위드,054920,3120,3315,3095,3285,426510,1378,6.31%,927,3090.019811,194.980189,2025.01.31
7,네오리진,094860,1058,1089,1005,1017,37333,38,-0.59%,218,1023.035938,-6.035938,2025.01.31
8,케이사인,192250,9100,9550,8790,9200,73956,680,1.1%,650,9099.900753,100.099247,2025.01.31
9,이스트소프트,047560,22900,24700,22450,24250,3136637,74882,11.24%,2816,21799.711257,2450.288743,2025.01.31


,회사,종가,전일대비,등락률,시가,고가,저가,거래량,거래대금(천),시가총액(억),날짜
0,SGA,315,2.003176,0.64%,317,325,315,48187,15,185,2025.01.31
1,SGA솔루션즈,492,2.983032,0.61%,484,504,484,60081,29,308,2025.01.31
2,SGA임베디드*(코스닥 종목),2965,0.000000,0,2965,2965,2965,0,0,161,-
3,안랩,76100,4798.582679,6.73%,71200,78300,71200,293959,22360,8467,2025.01.31
4,시큐브,920,5.032285,0.55%,911,927,909,110474,101,392,2025.01.31
5,윈스,11400,29.562621,0.26%,11370,11470,11190,19764,224,1400,2025.01.31
6,이글루,5090,10.159550,0.2%,5080,5320,4985,91341,466,560,2025.01.31
7,한컴위드,3285,194.980189,6.31%,3120,3315,3095,426510,1378,927,2025.01.31
8,네오리진,1017,-6.035938,-0.59%,1058,1089,1005,37333,38,218,2025.01.31
9,케이사인,9200,100.099247,1.1%,9100,9550,8790,73956,680,650,2025.01.31


In [4]:

from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import numbers

# 데이터프레임 전처리 함수
def preprocess_dataframe(df):
    for column in df.columns:
        if (column == "회사") or (column == "날짜"):  # 회사 이름은 변환하지 않음
            continue
        try:
            # 퍼센트 처리: '%'가 포함된 경우
            if df[column].astype(str).str.contains('%').any():
                continue

            else:
                # 숫자 변환
                df[column] = df[column].str.replace(',', '', regex=False).astype(float)
        except Exception as e:
            print(f"Error processing column {column}: {e}")
    return df


# 데이터프레임 전처리
df1 = preprocess_dataframe(df_num)
display(df1)

# 1. 서식화된 엑셀 파일 열기
formatted_file = "/content/drive/MyDrive/webcrawling/엑셀/SGA업계 및 경쟁사 현황4.xlsx"  # 서식화된 엑셀 파일 경로
output_file = "/content/drive/MyDrive/webcrawling/엑셀/SGA업계 및 경쟁사 현황_number_ktx.xlsx"  # 저장될 파일 경로

# 2. 기존 서식화된 엑셀 파일 불러오기
wb = load_workbook(formatted_file)
sheet = wb["작업용2"]  # 데이터 삽입할 시트 이름

# 3. 데이터 추가 위치 설정 (예: A3 셀부터 시작)
start_row = 2  # 데이터 시작 행
start_col = 2  # 데이터 시작 열 (1열 = 'A')

# 4. 데이터프레임에서 데이터 삽입
for row_index, row_data in enumerate(dataframe_to_rows(df1, index=False, header=True), start=start_row):
    for col_index, value in enumerate(row_data, start=start_col):
        cell = sheet.cell(row=row_index, column=col_index, value=value)

        # 숫자 데이터 처리
        if isinstance(value, (int, float)):
            cell.number_format = numbers.FORMAT_NUMBER  # 일반 숫자 형식

        # 퍼센트 데이터 처리
        elif isinstance(value, str) and "%" in value:
            try:
                # 퍼센트 값을 숫자로 변환 (예: "25%" -> 0.25)
                percent_value = float(value.strip('%'))/100
                cell.value = percent_value
                cell.number_format = numbers.FORMAT_PERCENTAGE  # 퍼센트 형식
            except ValueError:
                pass  # 변환 실패 시 문자열 그대로 저장


# 5. 엑셀 파일 저장
wb.save(output_file)
print(f"서식화된 엑셀 파일에 데이터가 추가되었습니다. 저장 파일: {output_file}")

Error processing column 종가: Can only use .str accessor with string values!
Error processing column 전일대비: Can only use .str accessor with string values!
Error processing column 시가: Can only use .str accessor with string values!
Error processing column 고가: Can only use .str accessor with string values!
Error processing column 저가: Can only use .str accessor with string values!
Error processing column 거래량: Can only use .str accessor with string values!
Error processing column 거래대금(천): Can only use .str accessor with string values!
Error processing column 시가총액(억): Can only use .str accessor with string values!


,회사,종가,전일대비,등락률,시가,고가,저가,거래량,거래대금(천),시가총액(억),날짜
0,SGA,315,2.003176,0.64%,317,325,315,48187,15,185,2025.01.31
1,SGA솔루션즈,492,2.983032,0.61%,484,504,484,60081,29,308,2025.01.31
2,SGA임베디드*(코스닥 종목),2965,0.000000,0,2965,2965,2965,0,0,161,-
3,안랩,76100,4798.582679,6.73%,71200,78300,71200,293959,22360,8467,2025.01.31
4,시큐브,920,5.032285,0.55%,911,927,909,110474,101,392,2025.01.31
5,윈스,11400,29.562621,0.26%,11370,11470,11190,19764,224,1400,2025.01.31
6,이글루,5090,10.159550,0.2%,5080,5320,4985,91341,466,560,2025.01.31
7,한컴위드,3285,194.980189,6.31%,3120,3315,3095,426510,1378,927,2025.01.31
8,네오리진,1017,-6.035938,-0.59%,1058,1089,1005,37333,38,218,2025.01.31
9,케이사인,9200,100.099247,1.1%,9100,9550,8790,73956,680,650,2025.01.31


서식화된 엑셀 파일에 데이터가 추가되었습니다. 저장 파일: /content/drive/MyDrive/webcrawling/엑셀/SGA업계 및 경쟁사 현황_number_ktx.xlsx


## 공시 정보 가져오기
- df_latest_disclosures

In [5]:
!pip install dart-fss

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.5/147.5 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.1/201.1 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 46.9 MB/s eta 0:00:00
  Attempting uninstall: termcolor
    Found existing installation: termcolor 2.5.0
    Uninstalling termcolor-2.5.0:
      Successfully uninstalled termcolor-2.5.0


In [6]:
import dart_fss as dart
import pandas as pd
import requests
import xml.etree.ElementTree as ET
import zipfile
import io
from bs4 import BeautifulSoup
api_key = 'ff12d16bfd2cb5ddef36494b71c419fda68b304d'
dart.set_api_key(api_key=api_key)

'ff12d16bfd2cb5ddef36494b71c419fda68b304d'

In [7]:
# 상장 기업명 크롤링
corp_list = dart.api.filings.get_corp_code()
corp_df = pd.DataFrame.from_dict(corp_list)
# corp_df = corp_df.dropna(subset = 'stock_code').sort_values('modify_date',ascending=False).reset_index(drop=True)
corp_df = pd.DataFrame.from_dict(corp_list).sort_values('modify_date', ascending=False).reset_index(drop=True)
corp_df['done_YN'] = "N"
corp_df

,corp_code,corp_name,stock_code,modify_date,done_YN
0,00922702,제이티비씨,None,20250131,N
1,01892046,리버앤씨,None,20250131,N
2,00104388,국도화학,007690,20250131,N
3,00390860,대산F&B,065150,20250131,N
4,00105855,엘에스일렉트릭,010120,20250131,N
...,...,...,...,...,...
109447,00295574,스톡캐스터,None,20170630,N
109448,00417547,스피어헤드,None,20170630,N
109449,00274298,신한산전,None,20170630,N
109450,00330886,쏘텍코리아,None,20170630,N


In [8]:
# 주어진 company_dic
company_dic = {"SGA": "049470", "SGA솔루션즈":"184230", "안랩":"053800", "시큐브":"131090", "윈스":"136540", "이글루":"067920",
               "한컴위드":"054920", "네오리진": "094860", "케이사인":"192250", "이스트소프트":"047560", "라온시큐어": "042510","파수":"150900",
               "한국정보인증": "053300", "지란지교시큐리티":"208350","수산아이앤티":"050960", "지니언스": "263860", "드림시큐리티": "203650",
               "모니터랩":"434480", "파이오링크":"170790", "시큐센":"232830", "시큐레터": "418250",
               "다우기술":"023590", "다우데이타":"032190", "한글과컴퓨터": "030520", "MDS테크":"086960", "아이티센":"124500", "콤텍시스템": "031820", "소프트센":"032680",
               "대신정보통신":"020180", "쌍용정보통신":"010280", "에스넷": "038680"
               }
# company_dic의 종목코드 리스트
target_stock_codes = list(company_dic.values())

# DataFrame에서 종목코드 필터링
filtered_df = corp_df[corp_df['stock_code'].isin(target_stock_codes)]

# 결과 출력
filtered_df

,corp_code,corp_name,stock_code,modify_date,done_YN
513,01038693,드림시큐리티,203650,20250102,N
686,00138303,쌍용정보통신,010280,20241220,N
754,00264635,에스넷,038680,20241217,N
1066,00577016,수산아이앤티,050960,20241129,N
3112,00402110,시큐브,131090,20240829,N
3782,00988364,SGA솔루션즈,184230,20240801,N
3788,00351579,SGA,049470,20240801,N
7881,00110875,대신정보통신,020180,20240522,N
7938,01583652,모니터랩,434480,20240520,N
11162,00186559,콤텍시스템,031820,20240402,N


In [9]:
# company_dic의 keys에 따라 순서를 맞춘 DataFrame 생성
sorted_filtered_df = pd.DataFrame(columns=filtered_df.columns)

for key in company_dic.keys():
    stock_code = company_dic[key]
    matching_row = filtered_df[filtered_df['stock_code'] == stock_code]
    if not matching_row.empty:
        sorted_filtered_df = pd.concat([sorted_filtered_df, matching_row])

# 결과 출력
first_df = sorted_filtered_df.iloc[:2]
voice_eye = corp_df[corp_df['corp_name']=='보이스아이']
rest_df = sorted_filtered_df.iloc[2:]
sorted_filtered_df = pd.concat([first_df, voice_eye,rest_df], ignore_index=True)
# sorted_filtered_df.to_csv("/content/drive/MyDrive/webcrawling/sorted_filtered_df.csv", index=False, encoding="utf-8-sig")


In [10]:
import pandas as pd
import requests
from datetime import datetime
from tqdm import tqdm  # 진행률 표시용 라이브러리

# target_date = '20241114'
# print(type(target_date))
# target_date = datetime.strptime(target_date, "%Y%m%d")
# print(type(target_date))
# print(target_date)

# API 인증키 및 기업 리스트
api_key = "ff12d16bfd2cb5ddef36494b71c419fda68b304d"  # OpenDART API 키 입력
comp_list = list(sorted_filtered_df['corp_code'])  # 조회할 기업 리스트

# 공시 데이터를 저장할 리스트
latest_disclosures = []

# OpenDART API URL
url_json = "https://opendart.fss.or.kr/api/list.json"

# 사용자 입력을 통해 특정 날짜 설정

print(f"📢 {target_date} 날짜의 공시 정보를 조회합니다...")

# 각 기업별 공시 데이터를 조회 (진행률 표시)
for corp_code in tqdm(comp_list, desc="🔍 기업별 공시 조회 중", unit="기업"):
    anchor = True
    params = {
        'crtfc_key': api_key,
        'corp_code': corp_code,
        'bgn_de': target_date  # 사용자가 입력한 날짜부터 조회 시작
    }

    try:
        response = requests.get(url_json, params=params)
        res = response.json()

        # API 응답 오류 처리
        if 'status' not in res or res['status'] == '013':  # 공시 데이터 없음
            latest_disclosures.append({
                'corp_code': corp_code,
                '공시 제목': '공시 없음',
                '공시 URL': ''
            })
            # continue

        # 공시 데이터가 있는 경우, 리스트 확인
        if 'list' in res:
            for disclosure in res['list']:  # 모든 공시 항목 확인
                rcept_date = disclosure['rcept_dt']  # 등재 날짜
                print(type(rcept_date))

                # 만약 공시 날짜가 조회 대상보다 이전이면 중단
                if rcept_date < target_date:
                    break

                # 목표 날짜의 공시만 저장
                if rcept_date == target_date:
                    title = disclosure['report_nm']
                    url = f"https://dart.fss.or.kr/dsaf001/main.do?rcpNo={disclosure['rcept_no']}"

                    latest_disclosures.append({
                        'corp_code': corp_code,
                        '공시 날짜': rcept_date,
                        '공시 제목': title,
                        '공시 URL': url
                    })
                    anchor = False
            if anchor == True:
              latest_disclosures.append({
                'corp_code': corp_code,
                '공시 제목': '공시 없음',
                '공시 URL': ''
            })
        # else:
        #     latest_disclosures.append({
        #         'corp_code': corp_code,
        #         '공시 제목': '공시 없음_2',
        #         '공시 URL': ''
        #     })

    except Exception as e:
        print(f"⚠️ {corp_code} 조회 중 오류 발생: {e}")
        latest_disclosures.append({
            'corp_code': corp_code,
            '공시 제목': '오류 발생',
            '공시 URL': ''
        })

# 결과를 데이터프레임으로 변환
df_latest_disclosures = pd.DataFrame(latest_disclosures)

# CSV 파일 저장
filename = f"disclosures_{target_date}.csv"
df_latest_disclosures.to_csv(filename, index=False, encoding="utf-8-sig")

# 결과 출력
display(df_latest_disclosures)
print(f"📂 {filename} 파일로 저장 완료!")



📢 20250131 날짜의 공시 정보를 조회합니다...


🔍 기업별 공시 조회 중:  19%|█▉        | 6/32 [00:06<00:27,  1.04s/기업]

<class 'str'>
<class 'str'>


🔍 기업별 공시 조회 중:  56%|█████▋    | 18/32 [00:18<00:14,  1.04s/기업]

<class 'str'>


🔍 기업별 공시 조회 중: 100%|██████████| 32/32 [00:33<00:00,  1.04s/기업]


,corp_code,공시 제목,공시 URL,공시 날짜
0,00351579,공시 없음,,NaN
1,00988364,공시 없음,,NaN
2,01621183,공시 없음,,NaN
3,00298270,공시 없음,,NaN
4,00402110,공시 없음,,NaN
5,00868705,영업(잠정)실적(공정공시),https://dart.fss.or.kr/dsaf001/main.do?rcpNo=2...,20250131
6,00868705,연결재무제표기준영업(잠정)실적(공정공시),https://dart.fss.or.kr/dsaf001/main.do?rcpNo=2...,20250131
7,00364847,공시 없음,,NaN
8,00363592,공시 없음,,NaN
9,00599106,공시 없음,,NaN


📂 disclosures_20250131.csv 파일로 저장 완료!


In [11]:
df_latest_disclosures = df_latest_disclosures[['공시 제목', '공시 URL']]
df_latest_disclosures

,공시 제목,공시 URL
0,공시 없음,
1,공시 없음,
2,공시 없음,
3,공시 없음,
4,공시 없음,
5,영업(잠정)실적(공정공시),https://dart.fss.or.kr/dsaf001/main.do?rcpNo=2...
6,연결재무제표기준영업(잠정)실적(공정공시),https://dart.fss.or.kr/dsaf001/main.do?rcpNo=2...
7,공시 없음,
8,공시 없음,
9,공시 없음,


In [12]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import Font

# 기존 엑셀 파일 경로
file_path = "/content/drive/MyDrive/webcrawling/엑셀/SGA업계 및 경쟁사 현황_number_ktx.xlsx"
output_path = f"SGA업계 및 경쟁사 현황_수치_공시_{target_date}.xlsx"

# 엑셀 파일 불러오기
wb = load_workbook(file_path)

# 작업용2 시트 가져오기 (없으면 생성)
if "경쟁사주가" in wb.sheetnames:
    ws = wb["경쟁사주가"]
else:
    ws = wb.create_sheet("경쟁사주가")

# 데이터 추가 시작 셀
start_row = 4  # 데이터 시작 행 번호
start_col = 11  # 데이터 시작 열 번호

# 현재 입력할 행 번호
current_row = start_row

# 데이터 추가
for row_idx, row in df_latest_disclosures.iterrows():
    excel_row_number = current_row


    if excel_row_number in [7,27]:
        current_row += 1  # 현재 행 번호를 1 증가시켜 다음 행으로 이동
    # 공시 제목이 '공시 없음'인 경우 스킵
    if row['공시 제목'] == '공시 없음':
        current_row += 1
        continue

    # 공시 제목 셀 추가 및 하이퍼링크 처리
    title_cell = ws.cell(row=current_row, column=start_col, value=row['공시 제목'])
    title_cell.hyperlink = row['공시 URL']
    title_cell.font = Font(color="0000FF", underline="single")

    # 다음 행으로 이동
    current_row += 1

# 엑셀 파일 저장
wb.save(output_path)
print(f"데이터프레임이 {output_path} 파일에 성공적으로 추가되었습니다.")

데이터프레임이 SGA업계 및 경쟁사 현황_수치_공시_20250131.xlsx 파일에 성공적으로 추가되었습니다.


# 뉴스 가져오기
- client id : IamYBKISQmZU5MdHRnsm
- client secret: AEZKqyZhux
-> 네이버 검색 api를 사용하여 해당 기업들의 뉴스를 최신 순으로 100개씩 가져오기
-> 그 후 내가 입력한 날짜와 뉴스 날짜가 맞고, title이나 description에 키워드 리스트에 있는 단어가 있고, avoid keyword list에 있는 단어가 포함되지 않으면 데이터프레임화한다(정확한 뉴스를 가져오기 위함)
- 네이버는 뉴스 검색 시 기사 제목(title)과 본문(description)에 검색 키워드가 포함된 뉴스를 우선적으로 반환

In [13]:
import urllib.request
import datetime
import json
import pandas as pd
from urllib.parse import quote
import re

# pd.set_option('display.max_rows', None)

# 네이버 API 인증 정보
client_id = "IamYBKISQmZU5MdHRnsm"
client_secret = "AEZKqyZhux"

# 검색할 기업 리스트 (기업명 -> 종목 코드 매핑)
company_dic = {
    "SGA": "049470", "SGA솔루션즈":"184230", "voiceye": "0", "안랩":"053800", "시큐브":"131090", "윈스":"136540", "이글루":"067920",
    "한컴위드":"054920", "네오리진": "094860", "케이사인":"192250", "이스트소프트":"047560", "라온시큐어": "042510","파수":"150900",
    "한국정보인증": "053300", "지란지교시큐리티":"208350","수산아이앤티":"050960", "지니언스": "263860", "드림시큐리티": "203650",
    "모니터랩":"434480", "파이오링크":"170790", "시큐센":"232830", "시큐레터": "418250",
    "다우기술":"023590", "다우데이타":"032190", "한글과컴퓨터": "030520", "MDS테크":"086960", "아이티센":"124500", "콤텍시스템": "031820", "소프트센":"032680",
    "대신정보통신":"020180", "쌍용정보통신":"010280", "에스넷": "038680"
}

# 포함할 키워드 리스트 (경제/IT 관련 키워드)
ECON_IT_KEYWORDS = ["경제", "금융", "산업", "투자", "주식", "증시", "기업", "소비",
                     "통화", "환율", "기술", "주가", "인터넷", "AI", "클라우드",
                    "데이터", "보안", '디지털', '코퍼레이션', '시장', '주주', '배당',
                    '등락', '플랫폼', '정보', '보안', '투자', '협약', '주', '솔루션',
                    '기관', '재무', 'IT', '상장', '공시']

# 제외할 키워드 리스트 (불필요한 뉴스 필터링)
EXCLUDE_KEYWORDS = ["연예", "스포츠", "날씨", "범죄", "게임", "드라마", "영화", "NBA", "축제", "추위", "호텔", "얼음"]

# 네이버 API 요청 함수
def getRequestUrl(url):
    req = urllib.request.Request(url)
    req.add_header("X-Naver-Client-Id", client_id)
    req.add_header("X-Naver-Client-Secret", client_secret)

    try:
        response = urllib.request.urlopen(req)
        if response.getcode() == 200:
            return response.read().decode('utf-8')
    except Exception as e:
        print(f"❌ Error: {e}")
        return None

# 네이버 뉴스 검색 API 호출 함수 (최대 100개만 가져오기)
def getNaverSearch(keyword):
    base = "https://openapi.naver.com/v1/search/news.json"
    parameters = f"?query={quote(keyword)}&start=1&display=100&sort=date"  # 최신순 정렬
    url = base + parameters

    response = getRequestUrl(url)
    if response is None:
        return None
    else:
        return json.loads(response)

# 뉴스 데이터 필터링 함수 (경제/IT 키워드 포함 여부 + 특정 키워드 제외 여부)
def filterEconomyITNews(post,company):
    """
    기사 제목이나 설명에 ECON_IT_KEYWORDS(경제/IT 키워드)가 포함되고,
    EXCLUDE_KEYWORDS(제외할 키워드)가 포함되지 않은 경우만 True 반환
    """
    title = post['title']
    description = post['description']

    # 포함할 키워드 체크 (하나라도 포함되면 OK)
    if not any(keyword in title or keyword in description for keyword in ECON_IT_KEYWORDS):
        return False  # 경제/IT 관련 기사가 아님

    if company == 'SGA':  # sga솔루션즈가 아닌 sga 관련 기사만을 가져오고 싶다
        if not (company in title or company in description):
            return False  # 'SGA'가 title 또는 description에 없으면 False 반환

        if re.search(r'SGA</b>\s*솔루션즈', title) or re.search(r'SGA</b>\s*솔루션즈', description):
            return False  # 'SGA' 뒤에 '솔루션즈'가 붙어 있으면 False 반환

    if not (company in title or company in description):
        return False  # description, title에 해당 기업의 이름이 들어가야

    # 제외할 키워드 체크 (하나라도 포함되면 False)
    if any(keyword in title or keyword in description for keyword in EXCLUDE_KEYWORDS):
        return False  # 제외할 키워드가 포함된 기사이므로 필터링

    return True  # 경제/IT 관련 기사이면서, 제외할 키워드가 포함되지 않은 경우

# 검색 실행 함수 (기업 리스트 전체 순회)
def main():
    # target_date = input("📅 검색할 날짜를 입력하세요 (YYYYMMDD): ").strip()

    all_news = []  # 모든 기업의 뉴스를 저장할 리스트
    total_cnt = 0  # 전체 뉴스 개수 카운트

    for company in company_dic.keys():
        jsonResult = []
        cnt = 0

        # API 요청
        jsonResponse = getNaverSearch(company)

        if jsonResponse is None or 'items' not in jsonResponse:
            print(f"❌ {company} 뉴스 검색 결과가 없습니다.")
            continue

        total = jsonResponse['total']
        print(f"🔎 {company} 전체 검색 결과: {total}건 (최대 100개 검색)")

        # 뉴스 가져오기 및 날짜 + 키워드 필터링
        for post in jsonResponse['items']:
            # 날짜 변환 (YYYYMMDD)
            pDate = datetime.datetime.strptime(post['pubDate'], '%a, %d %b %Y %H:%M:%S +0900')
            pDate = pDate.strftime('%Y%m%d')

            # 입력한 날짜와 일치하고, 경제/IT 키워드 포함 & 불필요한 뉴스 제외
            if pDate == target_date and filterEconomyITNews(post, company):
                cnt += 1
                jsonResult.append({
                    'cnt': cnt,
                    'company': company,  # 기업명 추가
                    'title': post['title'],
                    'description': post['description'],
                    'org_link': post['originallink'],
                    'link': post['link'],
                    'pDate': pDate
                })

        # 필터링된 뉴스만 리스트에 추가
        all_news.extend(jsonResult)
        total_cnt += len(jsonResult)

    # JSON 저장
    filename = f"CyberSecurity_News_{target_date}.json"
    with open(filename, 'w', encoding='utf8') as outfile:
        json.dump(all_news, outfile, indent=4, sort_keys=True, ensure_ascii=False)

    print(f"✅ 총 가져온 데이터: {total_cnt}건")
    print(f"📂 저장 완료: {filename}")

    # 결과를 DataFrame으로 변환하여 CSV 저장
    df = pd.DataFrame(all_news)
    if not df.empty:
        df.to_csv(f"CyberSecurity_News_{target_date}.csv", encoding='utf-8-sig', index=False)
        print(f"📂 CSV 저장 완료: CyberSecurity_News_{target_date}.csv")
        display(df)
    else:
        print("❌ 해당 날짜에 대한 경제/IT 뉴스가 없습니다.")

if __name__ == '__main__':
    main()


🔎 SGA 전체 검색 결과: 20299건 (최대 100개 검색)
🔎 SGA솔루션즈 전체 검색 결과: 7505건 (최대 100개 검색)
🔎 voiceye 전체 검색 결과: 160건 (최대 100개 검색)
🔎 안랩 전체 검색 결과: 80288건 (최대 100개 검색)
🔎 시큐브 전체 검색 결과: 8751건 (최대 100개 검색)
🔎 윈스 전체 검색 결과: 18561건 (최대 100개 검색)
🔎 이글루 전체 검색 결과: 29323건 (최대 100개 검색)
🔎 한컴위드 전체 검색 결과: 6939건 (최대 100개 검색)
🔎 네오리진 전체 검색 결과: 1694건 (최대 100개 검색)
🔎 케이사인 전체 검색 결과: 20945건 (최대 100개 검색)
🔎 이스트소프트 전체 검색 결과: 32372건 (최대 100개 검색)
🔎 라온시큐어 전체 검색 결과: 16885건 (최대 100개 검색)
🔎 파수 전체 검색 결과: 28490건 (최대 100개 검색)
🔎 한국정보인증 전체 검색 결과: 446280건 (최대 100개 검색)
🔎 지란지교시큐리티 전체 검색 결과: 7403건 (최대 100개 검색)
🔎 수산아이앤티 전체 검색 결과: 4342건 (최대 100개 검색)
🔎 지니언스 전체 검색 결과: 8467건 (최대 100개 검색)
🔎 드림시큐리티 전체 검색 결과: 8031건 (최대 100개 검색)
🔎 모니터랩 전체 검색 결과: 9566건 (최대 100개 검색)
🔎 파이오링크 전체 검색 결과: 8214건 (최대 100개 검색)
🔎 시큐센 전체 검색 결과: 3668건 (최대 100개 검색)
🔎 시큐레터 전체 검색 결과: 2820건 (최대 100개 검색)
🔎 다우기술 전체 검색 결과: 20290건 (최대 100개 검색)
🔎 다우데이타 전체 검색 결과: 15145건 (최대 100개 검색)
🔎 한글과컴퓨터 전체 검색 결과: 72891건 (최대 100개 검색)
🔎 MDS테크 전체 검색 결과: 6828건 (최대 100개 검색)
🔎 아이티센 전체 검색 결과: 15051건 (최대 100개 검색)
🔎

,cnt,company,title,description,org_link,link,pDate
0,1,SGA,'좀비기업 퇴출' 예고장…보안주 칼바람 부나,"동일한 기준으로 시큐센은 시총 281억원과 매출액 162억원, 소프트캠프는 시총 2...",https://www.ddaily.co.kr/page/view/20250131120...,https://n.news.naver.com/mnews/article/138/000...,20250131
1,2,SGA,[안티바이러스②] 보안의 기본으로 자리매김한 '안티바이러스',안랩 'V3' 외에도 △하우리 '바이로봇(ViRobot)' △잉카인터넷 '엔프로텍트...,http://www.itdaily.kr/news/articleView.html?id...,http://www.itdaily.kr/news/articleView.html?id...,20250131
2,3,SGA,한글과컴퓨터 주가 날라갈까... 인공지능 제품 3종 특징은?,한글과컴퓨터가 속한 클라우드 컴퓨팅 관련주(네이버 증권)에는 에스피소트트 한글과컴퓨...,https://www.pinpointnews.co.kr/news/articleVie...,https://www.pinpointnews.co.kr/news/articleVie...,20250131
3,1,SGA솔루션즈,IB역량 빛난 키움증권…&quot;올해 3건 스팩합병 상장&quot;[시그널],키움증권의 직전 스팩 합병 상장은 2015년 <b>SGA솔루션즈</b>(184230...,https://www.sedaily.com/NewsView/2GNWOM2C79,https://n.news.naver.com/mnews/article/011/000...,20250131
4,2,SGA솔루션즈,아진엑스텍 주가 '두둥실'...로봇 움직임 관련 제어기술 분야서 강점 부...,"삼익THK, <b>SGA솔루션즈</b>, 커머스마이너, NAVER, 삼성전자, 대성...",https://www.pinpointnews.co.kr/news/articleVie...,https://www.pinpointnews.co.kr/news/articleVie...,20250131
...,...,...,...,...,...,...,...
102,1,아이티센,유라클 주가 벌떡...자체 기술 개발 인공지능 챗봇 호평,유라클이 속한 STO(토큰증권 발행) 관련주(네이버 증권)에는 갤럭시아에스엠 <b...,https://www.pinpointnews.co.kr/news/articleVie...,https://www.pinpointnews.co.kr/news/articleVie...,20250131
103,1,쌍용정보통신,한글과컴퓨터 주가 날라갈까... 인공지능 제품 3종 특징은?,파이오링크 유엔젤 포스코DX 폴라리스오피스 포시에스 SK텔레콤 비투엔 효성ITX 엠...,https://www.pinpointnews.co.kr/news/articleVie...,https://www.pinpointnews.co.kr/news/articleVie...,20250131
104,1,에스넷,아진엑스텍 주가 '두둥실'...로봇 움직임 관련 제어기술 분야서 강점 부...,"증권)에는 <b>에스넷</b> 이스트소프트 DSC인베스트먼트, 포스코DX, 알서포트...",https://www.pinpointnews.co.kr/news/articleVie...,https://www.pinpointnews.co.kr/news/articleVie...,20250131
105,2,에스넷,링네트 주가 기지개...국방 네트워크 기술 선진화 기여 계획 부각,링네트가 속한 재택근무 스마트워크 관련주(네이버 증권)인 오파스넷 링네트 휴네시온 ...,https://www.pinpointnews.co.kr/news/articleVie...,https://www.pinpointnews.co.kr/news/articleVie...,20250131


In [14]:
df = pd.read_csv(f"/content/CyberSecurity_News_{target_date}.csv")

# display(df)
df_unique = df.drop_duplicates(subset=["title"], keep="first")
df_filtered = df_unique[~df_unique.duplicated(subset=['company'], keep='first')]
display(df_filtered)

,cnt,company,title,description,org_link,link,pDate
0,1,SGA,'좀비기업 퇴출' 예고장…보안주 칼바람 부나,"동일한 기준으로 시큐센은 시총 281억원과 매출액 162억원, 소프트캠프는 시총 2...",https://www.ddaily.co.kr/page/view/20250131120...,https://n.news.naver.com/mnews/article/138/000...,20250131
3,1,SGA솔루션즈,IB역량 빛난 키움증권…&quot;올해 3건 스팩합병 상장&quot;[시그널],키움증권의 직전 스팩 합병 상장은 2015년 <b>SGA솔루션즈</b>(184230...,https://www.sedaily.com/NewsView/2GNWOM2C79,https://n.news.naver.com/mnews/article/011/000...,20250131
10,1,안랩,2025년 전반기 최대 정보보호&amp;데이터보안 컨퍼런스…제13회 'ISDP 20...,방안-<b>안랩</b> 추상욱 부장- [B-1]클라우드 환경의 개인정보 보호 및 관...,https://www.dailysecu.com/news/articleView.htm...,https://www.dailysecu.com/news/articleView.htm...,20250131
21,1,한컴위드,<b>한컴위드</b> 주가 룰루랄라... '글로벌 방위산업 강소기업 육성사업' 과제...,<b>한컴위드</b> 주가가 강세를 보이고 있다. 31일 한국거래소에 따르면 이날 ...,https://www.pinpointnews.co.kr/news/articleVie...,https://www.pinpointnews.co.kr/news/articleVie...,20250131
28,1,이스트소프트,"<b>이스트소프트</b>, 국제구호개발 비정부기구 '더멋진세상'과 '페르소닷에이...",PERSO.ai로 한국어 IT 교육 콘텐츠를 프랑스어로 오토 더빙해 현지화한 영상 ...,https://www.aitimes.kr/news/articleView.html?i...,https://www.aitimes.kr/news/articleView.html?i...,20250131
63,1,라온시큐어,에이엘티 주가 불꽃랠리... '마이브 스타일폴더2(AT-M140)' 출시 호평,딥마인드 <b>라온시큐어</b> 한울소재과학 SK텔레콤 LG유플러스 한국첨단소재 아...,https://www.pinpointnews.co.kr/news/articleVie...,https://www.pinpointnews.co.kr/news/articleVie...,20250131
68,1,파수,"&quot;성능 좋아도 불안해서 못써요&quot;, 딥시크 오픈소스AI 한계 드러나나?",국내 보안 업체 <b>파수</b> 관계자는 &quot;딥시크 뿐 아니라 생성형 AI...,https://www.epnc.co.kr/news/articleView.html?i...,https://www.epnc.co.kr/news/articleView.html?i...,20250131
74,1,한국정보인증,유라클 주가 벌떡...자체 기술 개발 인공지능 챗봇 호평,뱅크웨어글로벌 교보증권 <b>한국정보인증</b> 우리기술투자 비유테크놀로지 아톤 아...,https://www.pinpointnews.co.kr/news/articleVie...,https://www.pinpointnews.co.kr/news/articleVie...,20250131
76,2,수산아이앤티,계엄에 정치 테마주 '들썩'... 이재명·한동훈·홍준표·김문수 등 관련...,"이외에도 에이텍(6일 지정), 동신건설(6일), 에이텍모빌리티(9일), 형지I&am...",https://www.pinpointnews.co.kr/news/articleVie...,https://www.pinpointnews.co.kr/news/articleVie...,20250131
81,3,드림시큐리티,클라우드 성장 둔화.. 마이크로소프트 6%대 급락,"종목명 : <b>드림시큐리티</b>(203650)▷현재가 : 3,480원(25.01...",https://news.mtn.co.kr/news-detail/20250131131...,https://news.mtn.co.kr/news-detail/20250131131...,20250131


##  기업 데이터(row)가 없다면 뉴스 없음으로 넣어주기

In [15]:
# 데이터프레임에 없는 기업 목록 찾기
missing_companies = [company for company in company_dic.keys() if company not in df_filtered["company"].values]

# 없는 기업들을 추가할 데이터프레임 생성
missing_data = pd.DataFrame({
    "cnt": 1,
    "company": missing_companies,
    "title": "뉴스 없음",
    "description": "뉴스 없음",
    "org_link": "뉴스 없음",
    "link": "뉴스 없음",
    "pDate": int(target_date)  # 기본 날짜 설정 (필요 시 수정 가능)
})

# 기존 데이터프레임과 합치기
df_filtered_all_comp = pd.concat([df_filtered, missing_data], ignore_index=True)
display(df_filtered_all_comp)

,cnt,company,title,description,org_link,link,pDate
0,1,SGA,'좀비기업 퇴출' 예고장…보안주 칼바람 부나,"동일한 기준으로 시큐센은 시총 281억원과 매출액 162억원, 소프트캠프는 시총 2...",https://www.ddaily.co.kr/page/view/20250131120...,https://n.news.naver.com/mnews/article/138/000...,20250131
1,1,SGA솔루션즈,IB역량 빛난 키움증권…&quot;올해 3건 스팩합병 상장&quot;[시그널],키움증권의 직전 스팩 합병 상장은 2015년 <b>SGA솔루션즈</b>(184230...,https://www.sedaily.com/NewsView/2GNWOM2C79,https://n.news.naver.com/mnews/article/011/000...,20250131
2,1,안랩,2025년 전반기 최대 정보보호&amp;데이터보안 컨퍼런스…제13회 'ISDP 20...,방안-<b>안랩</b> 추상욱 부장- [B-1]클라우드 환경의 개인정보 보호 및 관...,https://www.dailysecu.com/news/articleView.htm...,https://www.dailysecu.com/news/articleView.htm...,20250131
3,1,한컴위드,<b>한컴위드</b> 주가 룰루랄라... '글로벌 방위산업 강소기업 육성사업' 과제...,<b>한컴위드</b> 주가가 강세를 보이고 있다. 31일 한국거래소에 따르면 이날 ...,https://www.pinpointnews.co.kr/news/articleVie...,https://www.pinpointnews.co.kr/news/articleVie...,20250131
4,1,이스트소프트,"<b>이스트소프트</b>, 국제구호개발 비정부기구 '더멋진세상'과 '페르소닷에이...",PERSO.ai로 한국어 IT 교육 콘텐츠를 프랑스어로 오토 더빙해 현지화한 영상 ...,https://www.aitimes.kr/news/articleView.html?i...,https://www.aitimes.kr/news/articleView.html?i...,20250131
5,1,라온시큐어,에이엘티 주가 불꽃랠리... '마이브 스타일폴더2(AT-M140)' 출시 호평,딥마인드 <b>라온시큐어</b> 한울소재과학 SK텔레콤 LG유플러스 한국첨단소재 아...,https://www.pinpointnews.co.kr/news/articleVie...,https://www.pinpointnews.co.kr/news/articleVie...,20250131
6,1,파수,"&quot;성능 좋아도 불안해서 못써요&quot;, 딥시크 오픈소스AI 한계 드러나나?",국내 보안 업체 <b>파수</b> 관계자는 &quot;딥시크 뿐 아니라 생성형 AI...,https://www.epnc.co.kr/news/articleView.html?i...,https://www.epnc.co.kr/news/articleView.html?i...,20250131
7,1,한국정보인증,유라클 주가 벌떡...자체 기술 개발 인공지능 챗봇 호평,뱅크웨어글로벌 교보증권 <b>한국정보인증</b> 우리기술투자 비유테크놀로지 아톤 아...,https://www.pinpointnews.co.kr/news/articleVie...,https://www.pinpointnews.co.kr/news/articleVie...,20250131
8,2,수산아이앤티,계엄에 정치 테마주 '들썩'... 이재명·한동훈·홍준표·김문수 등 관련...,"이외에도 에이텍(6일 지정), 동신건설(6일), 에이텍모빌리티(9일), 형지I&am...",https://www.pinpointnews.co.kr/news/articleVie...,https://www.pinpointnews.co.kr/news/articleVie...,20250131
9,3,드림시큐리티,클라우드 성장 둔화.. 마이크로소프트 6%대 급락,"종목명 : <b>드림시큐리티</b>(203650)▷현재가 : 3,480원(25.01...",https://news.mtn.co.kr/news-detail/20250131131...,https://news.mtn.co.kr/news-detail/20250131131...,20250131


In [16]:
# 🔹 기업명을 company_dic.keys() 순서대로 정렬
df_filtered_all_comp["company"] = pd.Categorical(df_filtered_all_comp["company"], categories=company_dic.keys(), ordered=True)
df_filtered_all_comp = df_filtered_all_comp.sort_values("company").reset_index(drop=True)
display(df_filtered_all_comp)

,cnt,company,title,description,org_link,link,pDate
0,1,SGA,'좀비기업 퇴출' 예고장…보안주 칼바람 부나,"동일한 기준으로 시큐센은 시총 281억원과 매출액 162억원, 소프트캠프는 시총 2...",https://www.ddaily.co.kr/page/view/20250131120...,https://n.news.naver.com/mnews/article/138/000...,20250131
1,1,SGA솔루션즈,IB역량 빛난 키움증권…&quot;올해 3건 스팩합병 상장&quot;[시그널],키움증권의 직전 스팩 합병 상장은 2015년 <b>SGA솔루션즈</b>(184230...,https://www.sedaily.com/NewsView/2GNWOM2C79,https://n.news.naver.com/mnews/article/011/000...,20250131
2,1,voiceye,뉴스 없음,뉴스 없음,뉴스 없음,뉴스 없음,20250131
3,1,안랩,2025년 전반기 최대 정보보호&amp;데이터보안 컨퍼런스…제13회 'ISDP 20...,방안-<b>안랩</b> 추상욱 부장- [B-1]클라우드 환경의 개인정보 보호 및 관...,https://www.dailysecu.com/news/articleView.htm...,https://www.dailysecu.com/news/articleView.htm...,20250131
4,1,시큐브,뉴스 없음,뉴스 없음,뉴스 없음,뉴스 없음,20250131
5,1,윈스,뉴스 없음,뉴스 없음,뉴스 없음,뉴스 없음,20250131
6,1,이글루,뉴스 없음,뉴스 없음,뉴스 없음,뉴스 없음,20250131
7,1,한컴위드,<b>한컴위드</b> 주가 룰루랄라... '글로벌 방위산업 강소기업 육성사업' 과제...,<b>한컴위드</b> 주가가 강세를 보이고 있다. 31일 한국거래소에 따르면 이날 ...,https://www.pinpointnews.co.kr/news/articleVie...,https://www.pinpointnews.co.kr/news/articleVie...,20250131
8,1,네오리진,뉴스 없음,뉴스 없음,뉴스 없음,뉴스 없음,20250131
9,1,케이사인,뉴스 없음,뉴스 없음,뉴스 없음,뉴스 없음,20250131


In [17]:
!pip install newspaper3k
!pip install --upgrade lxml newspaper3k lxml_html_clean

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 45.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.9/104.9 kB 8.2 MB/s eta 0:00:00
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13539 sha256=50f82dcb38e677201271e355b52d242ad43561ae87cc5d036e9079370588c56a
  Stored in directory: /root/.cache/pip/wheels/fc/ab/f8/cce3a9ae6d828bd346be695f7ff54612cd22b7cbd7208d68f3
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-py3-none-any.whl size=3342 sha256=768a11140d8f166f0239e487ccc1f50a77da3c85585ca3f0b1e796d96c8a9e1f
  Stored in directory: /root/.cache/pip/wheels/80/d5/72/9cd9eccc819636436c6a6e59c22a0fb1ec

In [18]:
!pip install fuzzywuzzy

In [19]:
import pandas as pd
import re
from newspaper import Article
from fuzzywuzzy import fuzz

# ✅ 데이터프레임 예제 (실제 데이터프레임을 불러올 경우 아래 주석 처리 후 사용)
# df = pd.read_csv('your_data.csv')  # 기존 데이터 불러오기

# newspaper3k를 사용하여 기사 제목과 첫 3문장 추출하는 함수
def extract_news_data(url):
    try:
        article = Article(url, language='ko')
        article.download()
        article.parse()

        # 기사 제목 가져오기
        title = article.title.strip()

        # 기사 본문에서 첫 3문장 추출
        sentences = re.split(r'\.\s+|\n+', article.text)  # '.' 뒤 공백 또는 줄바꿈 기준 분리
        imp_sent = '. '.join(sentences[:3]) if len(sentences) >= 3 else '. '.join(sentences)  # 첫 3문장만 추출

        return title, imp_sent
    except Exception as e:
        print(f"❌ {url} 크롤링 오류: {e}")
        return "제목 없음", "본문 없음"

# ✅ 데이터프레임의 모든 뉴스 URL에 대해 `new_title`, `imp_sent` 컬럼 추가
df_filtered_all_comp[['new_title', 'imp_sent']] = df_filtered_all_comp['link'].apply(lambda x: pd.Series(extract_news_data(x)))

def update_title(row):
    similarity = fuzz.ratio(row["title"], row["new_title"])  # 문자열 유사도 계산 (0~100)
    return row["title"] if similarity < 80 else row["new_title"]  # 유사도가 80% 미만이면 title로 변경

df_filtered_all_comp["new_title"] = df_filtered_all_comp.apply(update_title, axis=1)
df_filtered_all_comp["new_title"] = df_filtered_all_comp["new_title"].str.replace(r"<\/?b>", "", regex=True)




# ✅ 결과 데이터프레임 출력
display(df_filtered_all_comp)  # 상위 5개 행 출력


/usr/local/lib/python3.11/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


❌ 뉴스 없음 크롤링 오류: Article `download()` failed with No connection adapters were found for ':/뉴스 없음' on URL :/뉴스 없음
❌ 뉴스 없음 크롤링 오류: Article `download()` failed with No connection adapters were found for ':/뉴스 없음' on URL :/뉴스 없음
❌ 뉴스 없음 크롤링 오류: Article `download()` failed with No connection adapters were found for ':/뉴스 없음' on URL :/뉴스 없음
❌ 뉴스 없음 크롤링 오류: Article `download()` failed with No connection adapters were found for ':/뉴스 없음' on URL :/뉴스 없음
❌ 뉴스 없음 크롤링 오류: Article `download()` failed with No connection adapters were found for ':/뉴스 없음' on URL :/뉴스 없음
❌ 뉴스 없음 크롤링 오류: Article `download()` failed with No connection adapters were found for ':/뉴스 없음' on URL :/뉴스 없음
❌ 뉴스 없음 크롤링 오류: Article `download()` failed with No connection adapters were found for ':/뉴스 없음' on URL :/뉴스 없음
❌ 뉴스 없음 크롤링 오류: Article `download()` failed with No connection adapters were found for ':/뉴스 없음' on URL :/뉴스 없음
❌ 뉴스 없음 크롤링 오류: Article `download()` failed with No connection adapters were found for ':/뉴스 없음' on URL 

,cnt,company,title,description,org_link,link,pDate,new_title,imp_sent
0,1,SGA,'좀비기업 퇴출' 예고장…보안주 칼바람 부나,"동일한 기준으로 시큐센은 시총 281억원과 매출액 162억원, 소프트캠프는 시총 2...",https://www.ddaily.co.kr/page/view/20250131120...,https://n.news.naver.com/mnews/article/138/000...,20250131,'좀비기업 퇴출' 예고장…보안주 칼바람 부나,"재생하기 재생시간 02:25. ""헌법 뜻에 따라 9인 체제 가동 우선""‥헌법학자 1..."
1,1,SGA솔루션즈,IB역량 빛난 키움증권…&quot;올해 3건 스팩합병 상장&quot;[시그널],키움증권의 직전 스팩 합병 상장은 2015년 <b>SGA솔루션즈</b>(184230...,https://www.sedaily.com/NewsView/2GNWOM2C79,https://n.news.naver.com/mnews/article/011/000...,20250131,"IB역량 빛난 키움증권…""올해 3건 스팩합병 상장""[시그널]","재생하기 재생시간 01:14. 한 대 놓치면 하염없이…""어머니만을 위해"" 아들의 선..."
2,1,voiceye,뉴스 없음,뉴스 없음,뉴스 없음,뉴스 없음,20250131,뉴스 없음,본문 없음
3,1,안랩,2025년 전반기 최대 정보보호&amp;데이터보안 컨퍼런스…제13회 'ISDP 20...,방안-<b>안랩</b> 추상욱 부장- [B-1]클라우드 환경의 개인정보 보호 및 관...,https://www.dailysecu.com/news/articleView.htm...,https://www.dailysecu.com/news/articleView.htm...,20250131,2025년 전반기 최대 정보보호&데이터보안 컨퍼런스…제13회 ‘ISDP 2025’ ...,"2월 11일 전국 공공, 금융, 기업 정보보호 담당자 1천여 명 이상 참석 예정. ..."
4,1,시큐브,뉴스 없음,뉴스 없음,뉴스 없음,뉴스 없음,20250131,뉴스 없음,본문 없음
5,1,윈스,뉴스 없음,뉴스 없음,뉴스 없음,뉴스 없음,20250131,뉴스 없음,본문 없음
6,1,이글루,뉴스 없음,뉴스 없음,뉴스 없음,뉴스 없음,20250131,뉴스 없음,본문 없음
7,1,한컴위드,<b>한컴위드</b> 주가 룰루랄라... '글로벌 방위산업 강소기업 육성사업' 과제...,<b>한컴위드</b> 주가가 강세를 보이고 있다. 31일 한국거래소에 따르면 이날 ...,https://www.pinpointnews.co.kr/news/articleVie...,https://www.pinpointnews.co.kr/news/articleVie...,20250131,한컴위드 주가 룰루랄라... '글로벌 방위산업 강소기업 육성사업' 과제 수행업체로 선정,네이버 증권. 한컴위드 주가가 강세를 보이고 있다. 31일 한국거래소에 따르면 이날...
8,1,네오리진,뉴스 없음,뉴스 없음,뉴스 없음,뉴스 없음,20250131,뉴스 없음,본문 없음
9,1,케이사인,뉴스 없음,뉴스 없음,뉴스 없음,뉴스 없음,20250131,뉴스 없음,본문 없음


In [20]:
news_excel_df = df_filtered_all_comp[['new_title', 'link']]
display(news_excel_df)

,new_title,link
0,'좀비기업 퇴출' 예고장…보안주 칼바람 부나,https://n.news.naver.com/mnews/article/138/000...
1,"IB역량 빛난 키움증권…""올해 3건 스팩합병 상장""[시그널]",https://n.news.naver.com/mnews/article/011/000...
2,뉴스 없음,뉴스 없음
3,2025년 전반기 최대 정보보호&데이터보안 컨퍼런스…제13회 ‘ISDP 2025’ ...,https://www.dailysecu.com/news/articleView.htm...
4,뉴스 없음,뉴스 없음
5,뉴스 없음,뉴스 없음
6,뉴스 없음,뉴스 없음
7,한컴위드 주가 룰루랄라... '글로벌 방위산업 강소기업 육성사업' 과제 수행업체로 선정,https://www.pinpointnews.co.kr/news/articleVie...
8,뉴스 없음,뉴스 없음
9,뉴스 없음,뉴스 없음


In [21]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import Font

# 기존 엑셀 파일 경로
file_path = f"SGA업계 및 경쟁사 현황_수치_공시_{target_date}.xlsx"
output_path = f"SGA업계 및 경쟁사 현황_수치_공시_뉴스{target_date}.xlsx"

# 엑셀 파일 불러오기
wb = load_workbook(file_path)

# 작업용2 시트 가져오기 (없으면 생성)
if "경쟁사주가" in wb.sheetnames:
    ws = wb["경쟁사주가"]
else:
    ws = wb.create_sheet("경쟁사주가")

# 데이터 추가 시작 셀
start_row = 4  # 데이터 시작 행 번호
start_col = 12  # 데이터 시작 열 번호

# 현재 입력할 행 번호
current_row = start_row

# 데이터 추가
for row_idx, row in news_excel_df.iterrows():
    excel_row_number = current_row


    if excel_row_number in [7,27]:
        current_row += 1  # 현재 행 번호를 1 증가시켜 다음 행으로 이동
    # 공시 제목이 '공시 없음'인 경우 스킵
    if row['new_title'] == '뉴스 없음':
        current_row += 1
        continue

    # 공시 제목 셀 추가 및 하이퍼링크 처리
    title_cell = ws.cell(row=current_row, column=start_col, value=row['new_title'])
    title_cell.hyperlink = row['link']
    title_cell.font = Font(color="0000FF", underline="single")

    # 다음 행으로 이동
    current_row += 1

# 엑셀 파일 저장
wb.save(output_path)
print(f"데이터프레임이 {output_path} 파일에 성공적으로 추가되었습니다.")

데이터프레임이 SGA업계 및 경쟁사 현황_수치_공시_뉴스20250131.xlsx 파일에 성공적으로 추가되었습니다.


In [22]:
df_filtered_all_comp['imp_sent'][0]

'재생하기 재생시간 02:25. "헌법 뜻에 따라 9인 체제 가동 우선"‥헌법학자 1백여 명 직격. ◀ 앵커 ▶ 이처럼 헌재 결정을 부정하는 권성동 원내대표의 주장에 대한 헌법학자들의 판단은 달랐습니다'

In [23]:
import pandas as pd

# 예제 데이터프레임 생성
data = {
    "company": ["SGA", "SGA", "SGA", "ABC", "ABC"],
    "title": ["Title A", "Title A", "Title B", "Title C", "Title C"],
    "description": ["Desc 1", "Desc 2", "Desc 3", "Desc 4", "Desc 5"],
    "org_link": ["link1", "link2", "link3", "link4", "link5"],
    "link": ["link1", "link2", "link3", "link4", "link5"],
    "pDate": [20250131, 20250131, 20250131, 20250131, 20250131]
}

df = pd.DataFrame(data)
display(df)

# 1. 같은 title을 가진 행 확인
duplicate_titles = df[df.duplicated(subset=["title"], keep=False)]

# 2. company별 데이터 개수 확인
company_counts = df["company"].value_counts()

# 3. 각 company가 2개 이상의 데이터를 가진 경우, 중복 title 중 첫 번째만 유지
filtered_df = df[df["company"].map(company_counts) < 2]  # company가 2개 미만이면 유지
df_filtered = df[df["company"].map(company_counts) >= 2].drop_duplicates(subset=["title"], keep="first")

# 최종 필터링된 데이터프레임 병합
final_df = pd.concat([filtered_df, df_filtered])
display(final_df)


,company,title,description,org_link,link,pDate
0,SGA,Title A,Desc 1,link1,link1,20250131
1,SGA,Title A,Desc 2,link2,link2,20250131
2,SGA,Title B,Desc 3,link3,link3,20250131
3,ABC,Title C,Desc 4,link4,link4,20250131
4,ABC,Title C,Desc 5,link5,link5,20250131


,company,title,description,org_link,link,pDate
0,SGA,Title A,Desc 1,link1,link1,20250131
2,SGA,Title B,Desc 3,link3,link3,20250131
3,ABC,Title C,Desc 4,link4,link4,20250131
